In [1]:
# this notebook is initiated from 1_Networks_Coding_TA_rev3_model-lg

from operator import itemgetter
import pdfplumber

%run ../_functions_rev2.ipynb

import spacy
from spacy import displacy
from spacy.matcher import Matcher
import spacy_curated_transformers

from collections import Counter
import re

import pickle

In [2]:
def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

tables = []
counter = 0
raw_text_cleaned = ''
TOI_start = 5
TOI_end = 23
with pdfplumber.open("../_inputs/S3 Tool Changer.pdf") as pdf:
#    lines = [[] for x in range(len(pdf.pages))]
    for page in pdf.pages[TOI_start-1:]:        
        tables.append(page.find_tables())
        table_bboxes = [i.bbox for i in tables[counter]]
        tables[counter] = [{'table': [[clean_text_3(x) if x is not None else x for x in item] for item in i.extract()], 'doctop': i.bbox[1]} for i in tables[counter]]
        non_table_words = [word for word in page.extract_words() if not any(
            [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]

        for cluster in pdfplumber.utils.cluster_objects(non_table_words+tables[counter], 'doctop', tolerance=5):
            if 'text' in cluster[0]:
                text = (' '.join([i['text'] for i in cluster]))+' '
                text = clean_text_3(text)
                raw_text_cleaned += text
#                lines[counter].append(' '.join([i['text'] for i in cluster]))
#            elif 'table' in cluster[0]:
#                lines[counter].append(cluster[0]['table'])
        counter+=1

In [3]:
nlp = spacy.load("en_core_web_lg")

In [4]:
doc = nlp(raw_text_cleaned)

In [5]:
sentence_spans = []
for sent in doc.sents:
    sentence_spans.append(sent)
sentence_spans

[XChange Sigma 3 Page 4 User`s Guide 3 SYSTEM DESCRIPTION,
 There are two (2) main components in the basic system: The XChange Robot Assembly One Robot Adaptor is needed for each manipulator changing tools.,
 The XChange Tooling Assemblies One Tooling Adaptor is needed for each tool.,
 The Sigma3 XChange System provides a fast and reliable method for a manipulator to connect to and release a number of Tooling Adaptor Assemblies (each equipped with different tooling).,
 Attachment is through a mechanical linkage.,
 To couple and release a Tooling Adaptor Assembly, air is ported to a double acting cylinder, which drives three cams.,
 As the cams extend and retract, the Tooling Adaptor Assembly is coupled or released from the Robot Assembly, allowing the units to physically connect and disconnect along with any utilities contained in attached modules.,
 Built into the housing of the Sigma 3 there are 21 electrical interface pins.,
 These are used for passing I/O signals from the tool, thr

In [39]:
counter = 0
for item in sentence_spans:
    print(counter, item)
    counter+=1

0 XChange Sigma 3 Page 4 User`s Guide 3 SYSTEM DESCRIPTION
1 There are two (2) main components in the basic system: The XChange Robot Assembly One Robot Adaptor is needed for each manipulator changing tools.
2 The XChange Tooling Assemblies One Tooling Adaptor is needed for each tool.
3 The Sigma3 XChange System provides a fast and reliable method for a manipulator to connect to and release a number of Tooling Adaptor Assemblies (each equipped with different tooling).
4 Attachment is through a mechanical linkage.
5 To couple and release a Tooling Adaptor Assembly, air is ported to a double acting cylinder, which drives three cams.
6 As the cams extend and retract, the Tooling Adaptor Assembly is coupled or released from the Robot Assembly, allowing the units to physically connect and disconnect along with any utilities contained in attached modules.
7 Built into the housing of the Sigma 3 there are 21 electrical interface pins.
8 These are used for passing I/O signals from the tool, th

In [6]:
phrase = 'tooling adaptor'
phraseParts = phrase.split()

phrase = r'(?=.*\b(' + phraseParts[0] + r'|tool)\b)(?=.*\b(' + phraseParts[1] + r'|adaptors)\b)(?!.*\b(assembly|assemblies|assembly`s)\b)'

In [12]:
phrase = r'(?=.*\b(tooling|tool)\b)(?=.*\b(adaptor|adaptors)\b)(?!.*\b(assembly|assemblies|assembly`s)\b)'

In [9]:
list_matched_sentss = []
counter=0
for item in sentence_spans:
    if re.search(phrase, item.text, re.IGNORECASE) or re.search(phrase, item.text, re.IGNORECASE) or re.search(phrase, item.text, re.IGNORECASE):
        #print(counter,"\n",item)
        counter+=1
        list_matched_sentss.append(item)
list_matched_sentss

[The XChange Tooling Assemblies One Tooling Adaptor is needed for each tool.,
 Tooling Adaptors can be electrically coded so the Robot Controller can verify acquisition of a specific Tooling Adaptor.,
 By installing a jumper wire between appropriate signal pins (via the mating electrical connector) you can produce a unique binary code for each Tooling Adaptor.,
 Or bolt up into the (6) 10mm tapped holes ( hardware supplied by customer) on the ISO 160mmdiameter bolt hole pattern located in the bottom of the Tooling Adaptor Unit.,
 Place the Robot Adaptor together with the Tool Adaptor, making sure the alignment pins are in the correct position.,
 2. Stop the motion when the distance between the Robot Adaptor Face and the Tooling Adaptor Face is approximately 1.5 mm or (.06 in).,
 6.1.2 Other Programming Considerations Tooling Adaptors may be coded to verify connection to the correct tool.,
 This may be accomplished by configuring the wiring such that two of the pins in the Tooling Adapt

In [10]:
list_matched_sents = list_matched_sentss

## filtration for Tooling Adaptor sents

#1 subsection title was included in the sentence because of missing punctuation 
list_matched_sents[0] = list_matched_sents[0][4:]

#2 bulletpoint number was included in the sentence
list_matched_sents[5] = list_matched_sents[5][2:]

#3 section title was included in the sentence because of missing punctuation
list_matched_sents[6] = list_matched_sents[6][4:]

#4 extracting the subsection whereas object name is present but not extracting
# its bulletpoints below since the object name is absent on those.
#list_matched_sents[9] = sentence_spans[154]
list_matched_sents = list_matched_sents[:-1]
#list_matched_sents.append(sentence_spans[156])
#list_matched_sents.append(sentence_spans[158])
#list_matched_sents.append(sentence_spans[160])
#list_matched_sents.append(sentence_spans[162])

list_matched_sents

[One Tooling Adaptor is needed for each tool.,
 Tooling Adaptors can be electrically coded so the Robot Controller can verify acquisition of a specific Tooling Adaptor.,
 By installing a jumper wire between appropriate signal pins (via the mating electrical connector) you can produce a unique binary code for each Tooling Adaptor.,
 Or bolt up into the (6) 10mm tapped holes ( hardware supplied by customer) on the ISO 160mmdiameter bolt hole pattern located in the bottom of the Tooling Adaptor Unit.,
 Place the Robot Adaptor together with the Tool Adaptor, making sure the alignment pins are in the correct position.,
 Stop the motion when the distance between the Robot Adaptor Face and the Tooling Adaptor Face is approximately 1.5 mm or (.06 in).,
 Tooling Adaptors may be coded to verify connection to the correct tool.,
 This may be accomplished by configuring the wiring such that two of the pins in the Tooling Adaptor are shorted together.,
 The number of leads needed to allow a unique c

In [12]:
#displacy.render(list_matched_sents[0], style="dep", jupyter=True, options={'distance':80})

In [11]:
def decisionMaker(sent):
    wordsInSents = {}
    
    for token in sent:
        if spacy.explain(token.pos_) in ('noun', 'pronoun', 'proper noun'):
            wordsInSents[token.text] = 'object'
        if spacy.explain(token.pos_) in ('particle', 'auxiliary', 'verb'):
            wordsInSents[token.text] = 'action'
        if spacy.explain(token.pos_) in ('adposition', 'subordinating conjunction', 'coordinating conjunction'):
            wordsInSents[token.text] = 'positioning'
        if spacy.explain(token.pos_) in ('determiner', 'adverb', 'adjective', 'numeral'):
            wordsInSents[token.text] = 'determinant'
    
    dummy = {}
    removals = []
    for i in range(1,len(wordsInSents)):
        if list(wordsInSents.values())[i] != list(wordsInSents.values())[i-1]:
            dummy[list(wordsInSents.keys())[i-1]] = list(wordsInSents.values())[i-1]
        else:
            dummy[list(wordsInSents.keys())[i-1]+' '+list(wordsInSents.keys())[i]] = list(wordsInSents.values())[i]
            removals.append(i)
    
    if list(wordsInSents.values())[-1] == list(wordsInSents.values())[-2]:
        dummy[list(wordsInSents.keys())[-2]+' '+list(wordsInSents.keys())[-1]] = list(wordsInSents.values())[-1]
        dummy.pop(list(wordsInSents.keys())[-2], None)
    else:
        dummy[list(wordsInSents.keys())[-1]] = list(wordsInSents.values())[-1]
    
    for k in removals:
        dummy.pop(list(wordsInSents.keys())[k], None)
    
    wordsInSents = dummy
    
    #for i in range(1, len(wordsInSents)):
    #    if list(wordsInSents.keys())[i] == list(wordsInSents.keys())[i-1][-len(list(wordsInSents.keys())[i]):]:
    #        wordsInSents.pop(list(wordsInSents.keys())[i], None)
    
    return wordsInSents

In [69]:
list_of_wordsInSent = []
for item in list_matched_sents:
    list_of_wordsInSent.append(decisionMaker(item))

In [84]:
with open('_inputs/list_of_wordsInSent_TA.pkl', 'wb') as f:
    pickle.dump(list_of_wordsInSent,f)

In [86]:
list_of_wordsInSent;

In [ ]:
#with open('saved_list.pkl', 'wb') as f:
#    pickle.dump(list_of_dicts, f)
        
#with open('saved_list.pkl', 'rb') as f:
#    loaded_list_of_dicts = pickle.load(f)